In [316]:
import os

In [317]:
%pwd

'f:\\Time Series Data Analysis (Upwork)\\end-to-end-time-series-data-analysis-with-mlflow'

In [318]:
#os.chdir('../')

In [319]:
%pwd

'f:\\Time Series Data Analysis (Upwork)\\end-to-end-time-series-data-analysis-with-mlflow'

In [320]:
import pandas as pd

In [321]:
jet_data=pd.read_csv('artifacts/data_ingestion/Dataset/train_FD001.txt', sep=" ", header=None)
jet_data.columns = ["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5"
                    ,"sensor6","sensor7","sensor8","sensor9","sensor10","sensor11","sensor12","sensor13"
                    ,"sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
                    ,"sensor20","sensor21","sensor22","sensor23"]

jet_data.head()

,id,cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,sensor22,sensor23
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN


In [322]:
jet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        20631 non-null  int64  
 1   cycle     20631 non-null  int64  
 2   op1       20631 non-null  float64
 3   op2       20631 non-null  float64
 4   op3       20631 non-null  float64
 5   sensor1   20631 non-null  float64
 6   sensor2   20631 non-null  float64
 7   sensor3   20631 non-null  float64
 8   sensor4   20631 non-null  float64
 9   sensor5   20631 non-null  float64
 10  sensor6   20631 non-null  float64
 11  sensor7   20631 non-null  float64
 12  sensor8   20631 non-null  float64
 13  sensor9   20631 non-null  float64
 14  sensor10  20631 non-null  float64
 15  sensor11  20631 non-null  float64
 16  sensor12  20631 non-null  float64
 17  sensor13  20631 non-null  float64
 18  sensor14  20631 non-null  float64
 19  sensor15  20631 non-null  float64
 20  sensor16  20631 non-null  fl

In [323]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [324]:
from src.TimeSeriesProject .constants import *
from src.TimeSeriesProject.utils.common import read_yaml, create_directories

In [325]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
        root_dir=config.root_dir,
        STATUS_FILE=config.STATUS_FILE,
        unzip_data_dir = config.unzip_data_dir,
        all_schema=schema,
        )

        return data_validation_config

In [326]:
import os
from src.TimeSeriesProject import logger

In [327]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def RUL_calculator(self,df, df_max_cycles):
        max_cycle = df_max_cycles["cycle"]
        result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='id', right_index=True)
        result_frame["RUL"] = result_frame["max_cycle"] - result_frame["cycle"]
        result_frame.drop(['max_cycle'], axis=1, inplace=True)
        return result_frame


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir,sep=" ", header=None)
            data.columns = ["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5"
                    ,"sensor6","sensor7","sensor8","sensor9","sensor10","sensor11","sensor12","sensor13"
                    ,"sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
                    ,"sensor20","sensor21","sensor22","sensor23"]
            
            data.drop(['sensor22', 'sensor23'], axis=1, inplace=True)
            data.to_csv('artifacts/data_validation/output_data.csv', index=False)

            jet_id_and_rul = data.groupby(['id'])[["id" ,"cycle"]].max()
            jet_id_and_rul.set_index('id', inplace=True)
            jet_id_and_rul.to_csv('artifacts/data_validation/jet_id_and_rul.csv', index=False)


            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            data = RUL_calculator(data, jet_id_and_rul)
            print(data.head())

            return validation_status
        
        except Exception as e:
            raise e

In [328]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    df = pd.read_csv('artifacts/data_validation/output_data.csv')
    df_max_cycles = pd.read_csv('artifacts/data_validation/jet_id_and_rul.csv')
    data_validation.RUL_calculator(df,df_max_cycles)
    data_validation.validate_all_columns()
    
except Exception as e:
    raise e

[2023-09-13 00:47:27,959: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-13 00:47:27,962: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-13 00:47:27,974: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-13 00:47:27,976: INFO: common: created directory at: artifacts]
[2023-09-13 00:47:27,978: INFO: common: created directory at: artifacts/data_validation]
   id  cycle     op1     op2    op3  sensor1  sensor2  sensor3  sensor4  \
0   1      1 -0.0007 -0.0004  100.0   518.67   641.82  1589.70  1400.60   
1   1      2  0.0019 -0.0003  100.0   518.67   642.15  1591.82  1403.14   
2   1      3 -0.0043  0.0003  100.0   518.67   642.35  1587.99  1404.20   
3   1      4  0.0007  0.0000  100.0   518.67   642.35  1582.79  1401.87   
4   1      5 -0.0019 -0.0002  100.0   518.67   642.37  1582.85  1406.22   

   sensor5  ...  sensor13  sensor14  sensor15  sensor16  sensor17  sensor18  \
0    14.62  ...   2388.02   8138.62    8.4195    